# NR PDSCH Throughput simulation
this example is similar to Matlat 5G toolbox [NR PDSCH Throughput example ](https://www.mathworks.com/help/5g/ug/nr-pdsch-throughput.html#NewRadioPDSCHThroughputExample-13) to measure NR PDSCH throughput

The example models these 5G NR features:

* DL-SCH transport channel coding

* PDSCH, PDSCH DM-RS
* 15Khz or 30KHz subcarrier spacing
* support wideband precoding, do not support subband precoding

Channel model support:
* tapped delay line (TDL) propagation channels and customized propagation channel
* MIMO channel: Uniform Linear Array and cross polarized antennas MIMO Correlation Matrices with [high,medium,low] correlation
    detailed information defined in TS38.101-4, TS38.104, TS36.101, TS36.104
* fractional timing error
* crystal oscillator error between transmitter and receiver which causes frequency error, timing error increasing,sampling errir
* doppler frequency

Receiver support:
* Perfect or practical synchronization and channel estimation
* channel estimation and timing and frequency estimation and compensation
* ZF/MMSE/MMSE-IRC/ML eualization algorithms
* Rank, PMI and CQI estimation

The figure shows the implemented processing chain, low-phy processing includes FFT/IFFT, DUC/DDC

``` mermaid
graph LR
A[DL-SCH] -->B(PDSCH and DMRS) --> C[Precoding] --> D[low-phy]-->D2[channel model]-->D3[low phy receiver]

-->D4[channel estimation]-->D5[PSCH decoding]-->D6[DL-SCH decoding]

```

## Simulation configuration

In [6]:
import numpy as np
num_of_sim = 2 #number of simulation
SNR_in_dB_list = np.arange(-4, 4, 0.5).tolist() # SNR range

In [7]:
import os
print(os.getcwd())
os.chdir('..')
print(os.getcwd())

/Users/jieliu/jie_github/python_5gtoolbox
/Users/jieliu/jie_github


## waveform and carrier  configuration

In [ ]:
import os
print(os.getcwd())
#first read default configuration
from scripts.internal import default_config_files
default_DL_config = default_config_files.read_DL_default_config_files()
waveform_config = default_DL_config["DL_waveform_config"]
carrier_config = default_DL_config["DL_carrier_config"]
pdsch_config = default_DL_config["pdsch_config"]  

#set waveform config
waveform_config["numofslots"] = 10 #waveform length 
waveform_config["samplerate_in_mhz"] = 30.72*8 #usually it is 30.72MHz*[4,8,16,32]

#set carrier config
carrier_config["BW"] = 40 #bandwidth in MHz
carrier_config["scs"] = 30 #15khz or 30KHz
carrier_config["PCI"] = 1 #physical layer cell identity in range 0 - 1007
carrier_config["carrier_frequency_in_mhz"] = 3840 #carrier central freq
carrier_config["num_of_ant"] = 1 #number of Tx antenna
carrier_config["Nr"] = 1 #number of Rx antenna
carrier_config["maxMIMO_layers"] = min(min(4,carrier_config["num_of_ant"]),carrier_config["Nr"])



/Users/jieliu/jie_github


FileNotFoundError: [Errno 2] No such file or directory: 'py5gphy/nr_default_config/default_DL_waveform_config.json'

## PDSCH configuration

In [ ]:
#set PDSCH config, pdsch_config_list contain one PDSCH config
pdsch_config["mcs_index"] = 10 
pdsch_config["num_of_layers"] = 1 #this value should be <= carrier_config["maxMIMO_layers"]

#PDSCH DMRS
pdsch_config["DMRS"]["NrOfDMRSSymbols"] = 1 #1: single symbol,  2: double symbol
pdsch_config["DMRS"]["NumCDMGroupsWithoutData"] = 1
pdsch_config["DMRS"]["DMRSAddPos"] = 1
pdsch_config["DMRS"]["PDSCHMappintType"] = "A"

#PDSCH resources
pdsch_config["StartSymbolIndex"] = 2
pdsch_config["NrOfSymbols"] = 12
pdsch_config["ResAlloType1"]["RBStart"] = 0
pdsch_config["ResAlloType1"]["RBSize"] = 100 #make sure this value <= carrier BW supported PRB size

#PDSCh codebook
pdsch_config["codebook"]["enable"] = False # no codebook based precoding
pdsch_config["precoding_matrix"] = []  #use identity matrix

#add pdsch_config to pdsch_config_list
pdsch_config_list = []
pdsch_config_list.append(pdsch_config)

## wireless channel configuration

In [ ]:
channel_model_config = default_config_files.read_default_channel_model_config()
channel_model_config["Nt"] = carrier_config["num_of_ant"] #Tx antenna number
channel_model_config["Nr"] = carrier_config["Nr"]         #Rx antenna number
channel_model_config["Timeoff_ns"] = 30 #Tx to Rx timing offset in ns
channel_model_config["rho"] = 0 #transmitter relative frequency error
channel_model_config["fm_inHz"] = 0 #maximum doppler frequency offset in Hz
channel_model_config["Rspat"] = [1] #MIMO correlation matrix, should be (Nt*Nr)X(Nt*Nr) matrix
channel_model_config["Pnoise_dB"] = -10 #noise power in dB
#"delay in ns, power in dB, Rayleigh or Rician, K in dB for Rician， fDo(doppler freq in hz) for Rician",
channel_model_config["multi_paths"] = [[0,0,"Rayleigh",0,0],[0,0,"Rician",13.3,10]]
channel_model_config["fi_inHz"] = carrier_config["carrier_frequency_in_mhz"] * 10**6 #"carrier frequency in Hz",
channel_model_config["fs_inHz"] = waveform_config["samplerate_in_mhz"] * 10**6 #time domain data sample rate in Hz
channel_model_config["scs"] = carrier_config["scs"]


## receiver configuration

In [ ]:
# channel estimation
Cest_methods = ["normal"]
channel_est_mode = "practical"  # option: ["perfect","practical"]
#channel equalization
Cequ_methods = ["ZF", "MMSE"] #options in ["ZF", "MMSE", "MMSE-IRC", "ML"]

## Processing


In [ ]:
from  py5gphy.channel_model import nr_channel_model
#first generate all testcases based on Cest methodm Cequ methods
testcases = []
for Cest_method in Cest_methods:
    for Cequ_method in Cequ_methods:
        testcases.append([Cest_method,Cequ_method])

#init

for snr_db in SNR_in_dB_list:
    nrChannelMode = nr_channel_model.NrChannelModel(channel_model_config,
    Pnoise_dB=snr_db, 
    fi_inHz=carrier_config["carrier_frequency_in_mhz"] * 10**6, #carrier frequency in Hz"
    fs_inHz = waveform_config["samplerate_in_mhz"] * 10**6, #time domain data sample rate in Hz
    scs =  carrier_config["scs"])
    
        #


NameError: name 'Rspat' is not defined